In [13]:
import os
from os.path import join as opj
import numpy as np
import random

import open3d as o3d
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt
import trimesh

from utils import *
from dataset import *

[001:832][28184] (stun_port.cc:96): Binding request timed out from 130.83.164.x:34738 (enp5s0)


In [14]:
import pickle as pkl
data_root = 'data_root'
new_aff_path = opj(data_root, 'remapped_full_shape_train_data.pkl')
with open(new_aff_path, 'rb') as f:
    new_aff_dataset = pkl.load(f)
print("Loaded dataset with {} samples".format(len(new_aff_dataset)))


Loaded dataset with 1096 samples


In [15]:
# pick a random datapoint
# set_random_seed(0)
shape_id = random.choice(list(new_aff_dataset.keys()))
datapoint = new_aff_dataset[shape_id]

In [16]:
print("Shape id ", shape_id, "\nSemantic class: ", datapoint['semantic_class'])
print("All affordances: ", datapoint["affordance_classes"])
print("Num affordances: ", len(datapoint["affordance_classes"]))

Shape id  dcec634f18e12427c2c72e575af174cd 
Semantic class:  Mug
All affordances:  ['handover', 'cut', 'stab', 'lift', 'wrap', 'pour', 'wear']
Num affordances:  7


In [17]:
pcl = datapoint['data']['coordinates']
# Viz pcl with o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcl)
# color black
pcd.paint_uniform_color([0, 0, 0])
# viz in jupyter nb
print("Drawing pcd with", len(pcl), "points")
draw(pcd)

Drawing pcd with 2048 points
[Open3D INFO] Window window_6 created.


WebVisualizer(window_uid='window_6')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open

In [18]:
# get pcls of all labels
labels_list = []
scores_list = []
for key in datapoint['data']['labels'].keys():
    labels = datapoint['data']['labels'][key]
    scores = labels.sum()
    print("Label: ", key, " Score: ", scores)
    labels_list.append(labels)
    scores_list.append(scores)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl)
    pcd.paint_uniform_color([0, 0, 0])
    np.asarray(pcd.colors)[labels.squeeze() > 0] = np.random.uniform(0, 1, 3)
    print("Label 0: ", key)
    draw([pcd])

Label:  handover  Score:  199.07465
Label 0:  handover
[Open3D INFO] Window window_7 created.


WebVisualizer(window_uid='window_7')

Label:  pour  Score:  68.00398
Label 0:  pour
[Open3D INFO] Window window_8 created.


WebVisualizer(window_uid='window_8')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.9002426487556947
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.9002426487556947
[Open3D INFO] Sending init frames to window_6.
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP AP